In [1]:
import os
import numpy as np
import Model

from config import root, dir_dataset_test

import warnings
warnings.filterwarnings("ignore")

In [2]:
list_of_model_name = [
    'v1-resnet-50-fer2013',
    'v2-resnet-50-personal'
]

In [3]:
def create_index(source):
    return (source, os.listdir(source))

def generator(source, index, num_batch, data_augmentation=True):
    while(True):
        np.random.shuffle(index)
        for i in range(num_batch):
            X, y = np.load( os.path.join(source, index[i]), allow_pickle=True )
            X = np.asarray( list(X), 'float32' )
            if(data_augmentation):
                X = Model.data_augmentation(X)
            y = np.asarray( list(y), 'float32' )
            yield(X, y)    

def create_generator(index, data_augmentation=True):
    source = index[0]
    index = index[1]
    num_batch = len(index)
    gen = generator(source, index, num_batch, data_augmentation)
    return gen, num_batch

def evaluate(model_path, gen, num_batch):
    model = Model.load_model(model_path)
    result = model.evaluate_generator(
        generator = gen,
        steps = num_batch,
        max_queue_size = 20,
        verbose = 0,
    )
    return result

In [4]:
test_index = create_index(dir_dataset_test)
test_gen, test_num_batch = create_generator(test_index, data_augmentation=False)

In [5]:
for model_name in list_of_model_name:
    path_model = os.path.join(root, 'Model/', model_name)
    result = evaluate(path_model, test_gen, test_num_batch)
    
    print('Model : %s'%(model_name))
    print('Loss  : %.6f'%(result[0]))
    print('Acc   : %.4f%%'%(result[1]*100))
    print()

Successfully loaded model from ../Model/v1-resnet-50-fer2013
Model : v1-resnet-50-fer2013
Loss  : 1.189698
Acc   : 55.5308%

Successfully loaded model from ../Model/v2-resnet-50-personal
Model : v2-resnet-50-personal
Loss  : 3.238765
Acc   : 29.5486%

